In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt

In [2]:
df = pd.read_csv("../Datasets/flavornet_dataset.csv")


In [8]:
df = df.dropna() # Drop N/A values
print("Number of odorants is %d" % len(df))
print("\n")
print("Sample of the dataset")
print(df.head())

Number of odorants is 634


Sample of the dataset
          Odorant                      Odor SMILES representation    exactmw  \
0  trimethylamine                      fish                CN(C)C  59.073499   
1         ethanal            pungent, ether                  CC=O  44.026215   
2    methanethiol  sulfur, gasoline, garlic                    CS  48.003371   
3        propanal          solvent, pungent                 CCC=O  58.041865   
4         pentane                    alkane                 CCCCC  72.093900   

   lipinskiHBA  lipinskiHBD  NumRotatableBonds  NumHBD  NumHBA  \
0          1.0          0.0                0.0     0.0     1.0   
1          1.0          0.0                0.0     0.0     1.0   
2          0.0          0.0                0.0     1.0     1.0   
3          1.0          0.0                1.0     0.0     1.0   
4          0.0          0.0                2.0     0.0     0.0   

   NumHeteroatoms  ...  NumSaturatedHeterocycles  NumAliphaticHeterocycl

In [ ]:
# Graph to view the distribution of number of descriptors per molecule
